Some times more than generic searches, we would like to exploit the fact that we count with a trustful resource, a curated knowledge that the agent can leverage.

In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

A simplified version of the previous example using Agno and LanceDB may show the potential of higher level frameworks.

### Vector database

[LanceDB](https://www.lancedb.com/) is a simple solution for tests and trials as it offers both, Cloud hosted and on-premises OSS solutions.

Let's first select a site to load data from. For example, some site documentation https://python.langchain.com/docs/introduction/

In [2]:
from agno.embedder.google import GeminiEmbedder
from agno.vectordb.lancedb import LanceDb, SearchType
from agno.knowledge.markdown import MarkdownKnowledgeBase

# Database
vector_db = LanceDb(
    table_name="langchain",
    uri="./lancedb",  # You can change this path to store data elsewhere
    embedder=GeminiEmbedder(),
    use_tantivy=True
)

# Knowledge base
knowledge_base = MarkdownKnowledgeBase(
    path="../markdown_files",
    vector_db=vector_db,
    search_type=SearchType.hybrid
)

INFO Creating table: langchain

[2025-08-01T08:58:23Z WARN  lance::dataset::write::insert] No existing dataset at /home/iraitz/Documentos/Projects/Agents/examples/lancedb/langchain.lance, it will be created


You can check how the embedder does its magic against the knowledge base.

In [3]:
query = {
    "text" : "What are agents?",
    "vector" : GeminiEmbedder().get_embedding("What are agents?")
}
query['vector'][:10]

[-0.025601903,
 -0.015883127,
 -0.006648923,
 -0.057227448,
 -0.007338925,
 0.021304669,
 -0.00434841,
 0.010607597,
 -0.0051602297,
 0.030773513]

We might need to load the knowledge base before querying.

In [4]:
knowledge_base.load()

INFO Loading knowledge base

INFO Reading: ../markdown_files/agents.md

INFO Added 3 documents to knowledge base

Of course, some tools we might need.

In [5]:
from agno.tools.tavily import TavilyTools

# Search engine
search = TavilyTools()

Now we can equip our agent with those options.

In [6]:
from agno.agent import Agent
from agno.models.google import Gemini

# Create an Agent
agent = Agent(
    model=Gemini(id="gemini-2.5-flash", temperature=0),
    knowledge=knowledge_base,
    tools=[search],
    # search_knowledge=True gives the Agent the ability to search on demand
    # search_knowledge is True by default
    search_knowledge=True,
    instructions=[
        "Include sources in your response.",
        "Always search your knowledge before answering the question.",
        "Only include the output in your response. No other text.",
    ],
    show_tool_calls=True,
    markdown=True,
    add_references=True,
)

Ok, we should see if the knowledge base is ready to be searched.

In [7]:
if agent.knowledge is not None:  # Check if knowledge base exists
    agent.knowledge.load()  # Load and index the PDF content

INFO Loading knowledge base

INFO Reading: ../markdown_files/agents.md

INFO Skipped 3 existing/duplicate documents.

INFO Added 0 documents to knowledge base

In [8]:
agent.print_response("What are agents?", stream=True)

Output()

INFO Found 3 documents

It is a small knowledge base but it made its magic.